<a href="https://colab.research.google.com/github/elangbijak4/Security_Research/blob/main/Tool43_Almudaya_Deteksi_Aktivitas_menggunakan_Log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Buat data log simulasi

In [1]:
import pandas as pd
import numpy as np
import random

# Buat data log contoh
np.random.seed(0)

# Contoh IP
ips = [f"192.168.1.{i}" for i in range(1, 101)] + ["203.0.113.1", "203.0.113.2"]

# Contoh Port
ports = [22, 80, 443, 8080, 3306, 5000, 22, 22, 22, 22, 22, 5000, 5000, 5000, 5000, 5000] + list(range(1000, 1010))

# Jenis Aktivitas
activities = ['login', 'logout', 'data_transfer', 'failed_login']

# Buat log contoh
log_data = []

for _ in range(1000):
    ip = random.choice(ips)
    port = random.choice(ports)
    activity = random.choice(activities)
    timestamp = pd.Timestamp.now() - pd.to_timedelta(random.randint(0, 3600), unit='s')
    log_data.append([ip, port, activity, timestamp])

# DataFrame log
log_df = pd.DataFrame(log_data, columns=['IP', 'Port', 'Activity', 'Timestamp'])

# Tampilkan beberapa baris
log_df.head()

,IP,Port,Activity,Timestamp
0,192.168.1.96,1002,logout,2024-08-01 06:38:51.604005
1,192.168.1.31,1002,login,2024-08-01 06:46:45.605478
2,192.168.1.80,1008,failed_login,2024-08-01 06:47:16.605730
3,192.168.1.43,3306,data_transfer,2024-08-01 06:11:29.605848
4,192.168.1.13,5000,data_transfer,2024-08-01 06:45:40.605932


Deteksi Aksi Mencurigakan

In [2]:
from collections import Counter

# Deteksi Port Scanning
port_scanning_threshold = 10
ip_port_count = log_df.groupby('IP')['Port'].nunique()
port_scanners = ip_port_count[ip_port_count > port_scanning_threshold]
print("Port Scanners Detected:")
print(port_scanners)

# Deteksi Brute Force Attacks
brute_force_threshold = 5
failed_logins = log_df[log_df['Activity'] == 'failed_login']
failed_login_count = failed_logins.groupby(['IP', failed_logins['Timestamp'].dt.date]).size()
brute_forcers = failed_login_count[failed_login_count > brute_force_threshold]
print("\nBrute Force Attacks Detected:")
print(brute_forcers)

# Deteksi Unusual Traffic Patterns
traffic_threshold = 20
traffic_pattern = log_df.groupby(['IP', log_df['Timestamp'].dt.hour]).size()
unusual_traffic = traffic_pattern[traffic_pattern > traffic_threshold]
print("\nUnusual Traffic Patterns Detected:")
print(unusual_traffic)

# Deteksi Access from Suspicious IPs
suspicious_ips = ['203.0.113.1', '203.0.113.2']
suspicious_access = log_df[log_df['IP'].isin(suspicious_ips)]
print("\nAccess from Suspicious IPs Detected:")
print(suspicious_access)

Port Scanners Detected:
IP
192.168.1.50    11
192.168.1.55    12
Name: Port, dtype: int64

Brute Force Attacks Detected:
IP            Timestamp 
192.168.1.10  2024-08-01    7
192.168.1.11  2024-08-01    6
192.168.1.47  2024-08-01    8
192.168.1.59  2024-08-01    7
192.168.1.84  2024-08-01    6
dtype: int64

Unusual Traffic Patterns Detected:
Series([], dtype: int64)

Access from Suspicious IPs Detected:
              IP  Port       Activity                  Timestamp
62   203.0.113.2    22         logout 2024-08-01 07:00:48.607845
134  203.0.113.2  5000          login 2024-08-01 06:51:20.609858
224  203.0.113.2  5000          login 2024-08-01 06:17:47.612185
264  203.0.113.2  1009   failed_login 2024-08-01 06:44:41.613235
463  203.0.113.2  1004  data_transfer 2024-08-01 07:01:45.619199
542  203.0.113.1  1001          login 2024-08-01 06:17:28.621202
689  203.0.113.2  5000   failed_login 2024-08-01 07:04:54.626507
800  203.0.113.1  1000          login 2024-08-01 06:10:57.631324
977  20